In [1]:
import os

import cv2
import numpy as np
from dotenv import load_dotenv

# Classification for the base classes

load_dotenv()

# Specify the path to the dataset directory
dataset_dir = os.getenv('DATASET_DIR')

target_size = (128, 128)

In [2]:
# Get all images
def read_images_from_directory(directory):
    images = []
    labels = []
    class_names = []
    path = []
    for root, dirs, files in os.walk(directory):
        for class_name in dirs:
            class_directory = os.path.join(root, class_name)
            for filename in os.listdir(class_directory):
                file_path = os.path.join(class_directory, filename)
                if os.path.isfile(file_path):
                    image = cv2.imread(file_path, cv2.COLOR_BGR2GRAY)
                    if image is not None:
                        image = cv2.resize(image, target_size)
                        images.append(image)
                        labels.append(os.path.basename(root) + "." + class_name)
                        path.append(file_path)
                        if class_name not in class_names:
                            class_names.append(class_name)

    images = np.array(images)
    labels = np.array(labels)
    class_names = np.array(class_names)
    path = np.array(path)

    return images, labels, class_names, path

In [3]:
images, labels, class_names, path = read_images_from_directory(dataset_dir)
images_with_labels = list(zip(images, labels))
print("Number of images:", len(images))
print("Number of labels:", len(labels))
print("Class names:", class_names)
print("Labels:", labels)

Number of images: 21
Number of labels: 21
Class names: ['big' 'small' 'agriculture_vehicle' 'bus' 'sportscar' 'curvy' 'adult'
 'child' 'handicaped' 'senior' 'danger' 'information' 'restriction'
 'speed_limit' 'yield' 'green' 'red' 'yellow']
Labels: ['animal.big' 'animal.small' 'car.agriculture_vehicle' 'car.bus'
 'car.sportscar' 'lane.curvy' 'pedestrian.adult' 'pedestrian.child'
 'pedestrian.handicaped' 'pedestrian.handicaped' 'pedestrian.senior'
 'traffic_sign.danger' 'traffic_sign.information'
 'traffic_sign.restriction' 'traffic_sign.restriction'
 'traffic_sign.restriction' 'traffic_sign.speed_limit'
 'traffic_sign.yield' 'trafic_light.green' 'trafic_light.red'
 'trafic_light.yellow']


In [4]:
print("First image: ", images_with_labels[0][0])
print("First image - label: ", images_with_labels[0][1])

First image:  [[[ 18  36  44]
  [ 11  30  34]
  [ 12  31  35]
  ...
  [  7  19  27]
  [  9  21  29]
  [ 15  26  34]]

 [[ 13  30  33]
  [ 12  39  45]
  [ 10  39  44]
  ...
  [ 10  20  27]
  [  9  21  29]
  [ 14  26  32]]

 [[ 11  32  34]
  [ 21  57  60]
  [ 20  56  62]
  ...
  [ 11  22  36]
  [ 11  23  32]
  [ 11  23  33]]

 ...

 [[ 66 149 142]
  [ 75 163 158]
  [ 64 148 146]
  ...
  [ 40  99  83]
  [ 44 106  93]
  [ 33  89  78]]

 [[ 62 147 140]
  [ 59 149 142]
  [ 63 156 144]
  ...
  [ 54 102  96]
  [ 17  82  71]
  [ 66 132 133]]

 [[ 68 152 141]
  [ 60 146 138]
  [ 77 151 154]
  ...
  [ 41 111 103]
  [ 50 118 113]
  [ 45 117 110]]]
First image - label:  animal.big


In [16]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

def annotate_image(image_path):
    # Load the pre-trained VGG16 model
    model = VGG16()

    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (224, 224))
    img_preprocessed = preprocess_input(np.expand_dims(img_resized, axis=0))

    # Make predictions on the preprocessed image
    predictions = model.predict(img_preprocessed)
    #     print(predictions)
    #     input('p a k ...')
    decoded_predictions = decode_predictions(predictions, top=3)[0]

    # Annotate the image with rectangles around the predicted objects
    annotated_img = img.copy()
    for _, label, prob in decoded_predictions:
        label = label.replace('_', ' ')
        print(label)
    # # Display and save the annotated image
    # cv2.imshow("Annotated Image", annotated_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.imwrite("annotated_image.jpg", annotated_img)

for i in range(0, len(path)):
    print(labels[i])
    annotate_image(path[i])

animal.big
1/1 [==============================] - 2s 2s/step
gazelle
impala
hartebeest
animal.small
1/1 [==============================] - 0s 451ms/step
weasel
badger
skunk
car.agriculture_vehicle
1/1 [==============================] - 1s 592ms/step
tractor
harvester
plow
car.bus
1/1 [==============================] - 1s 650ms/step
streetcar
trolleybus
passenger car
car.sportscar
1/1 [==============================] - 1s 638ms/step
sports car
racer
convertible
lane.curvy
1/1 [==============================] - 1s 559ms/step
dam
valley
promontory
pedestrian.adult
1/1 [==============================] - 0s 435ms/step
web site
jean
pajama
pedestrian.child
1/1 [==============================] - 0s 481ms/step
umbrella
jean
balance beam
pedestrian.handicaped
1/1 [==============================] - 1s 665ms/step
jinrikisha
rocking chair
shopping cart
pedestrian.handicaped
1/1 [==============================] - 0s 470ms/step
backpack
swimming trunks
academic gown
pedestrian.senior
1/1 [==========

In [12]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_images, test_images, train_labels0, test_labels0 = train_test_split(images, labels, test_size=0.2,
                                                                          random_state=42)

# Convert the labels to one-hot encoded vectors
num_classes = len(np.unique(labels))

print(num_classes)

label_encoder = LabelEncoder()
train_labels1 = label_encoder.fit_transform(train_labels0)
test_labels1 = label_encoder.fit_transform(test_labels0)

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels1, num_classes)
test_labels = to_categorical(test_labels1, num_classes)

18


In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional and pooling layers
# Fixme
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more convolutional and pooling layers if desired

# Flatten the output from the previous layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
num_epochs = 1
batch_size = 10

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(16, 128, 128, 3)
(16, 18)
(5, 128, 128, 3)
(5, 18)


In [10]:
# Train the model
model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size,
          validation_data=(test_images, test_labels))

2/2 [==============================] - 2s 532ms/step - loss: 1932.4266 - accuracy: 0.0000e+00 - val_loss: 5062.2324 - val_accuracy: 0.0000e+00


In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 33ms/step - loss: 5062.2324 - accuracy: 0.0000e+00
Test Loss: 5062.2324
Test Accuracy: 0.0000
